In [10]:
import os
import yaml
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain import hub
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings


# Load OPENAI & Langsmith API Keys
print("Loading Configuration data")

def load_config(configfile):
    with open(configfile, 'r') as file:
        config = yaml.safe_load(file)
        return config

config = load_config('apikeys.yaml')

ASTRA_DB_APPLICATION_TOKEN = config['apikeys']['astra_vector_db']
ASTRA_DB_API_ENDPOINT = "https://3787f08e-f351-42f4-b3bd-cfb6f9f924b2-us-east1.apps.astra.datastax.com"
ASTRA_DB_KEYSPACE = "default_keyspace"
LANGCHAIN_API_KEY = config['apikeys']['langsmith']
OPENAI_API_KEY = config['apikeys']['openai']

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Initialize the LLM
llm = ChatOpenAI(model='gpt-4o')

print("Loading the blog contents") 
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://www.jeyamohan.in/200801/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("td-post-content tagdiv-type",)
        )
    ),
)

docs = loader.load()

print("Splitting the blog content into smaller chunks")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Retrieve and generate using the relevant snippets of the blog

embeddings = OpenAIEmbeddings()

vstore = AstraDBVectorStore(
    embedding = embeddings,
    namespace = ASTRA_DB_KEYSPACE,
    collection_name="docs",
    token = ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint = ASTRA_DB_API_ENDPOINT
)

inserted_ids = vstore.add_documents(splits)
print(f"\nInserted {len(inserted_ids)} documents.")

retriever = vstore.as_retriever()
prompt = hub.pull("rag_detailed")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Loading Configuration data
Loading the blog contents
Splitting the blog content into smaller chunks

Inserted 7 documents.


In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'rag_detailed', 'lc_hub_commit_hash': '9f9d46ca7e12e0ec2b305631beebd191f5f7b35150f79bfba731a867a2b22a7a'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If the answer cannot be found in the context, just say that you don't know. If you don't know the answer, just say that you don't know. Be as detailed as possible with the answers.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [6]:
print(rag_chain.invoke("நம் கல்விமுறையின் அடிப்படைச் சிக்கல்?"))

நம் கல்விமுறையின் அடிப்படைச் சிக்கல் என்னவென்றால், சுயசிந்தனை (அல்லது சுயமாக சிந்தித்து, புதிய கருத்துகளை உருவாக்குதல்) கற்பிக்கப்படுவதில்லை. மாணவர்கள் கற்றுக்கொள்வது முழுமையாக புத்தகங்களில் உள்ளவற்றை நினைவில் கொண்டு எழுதுவதையே ஆகும். இது அவர்களை சிந்திக்காமல், நினைவில் வைத்துக்கொள்ளும் இயந்திரங்களாக மாற்றுகிறது. மேலும், இந்தியக் கல்விமுறையின் மற்றொரு முக்கிய சிக்கல், மதிப்பெண்களை வழங்குவதில் வெளிப்படைத்தன்மையை பேணுவதும், தவறுகளை நிரூபிப்பதில் உள்ள சிரமமும் ஆகும். இதனால் மாணவர்கள் தங்கள் கருத்தை வெளிப்படுத்தாதவர்களாகவும், சுருக்கமாக சிந்திக்கத் தெரியாதவர்களாகவும் உருவாகின்றனர்.


In [7]:
print(rag_chain.invoke("தத்துவப்படுத்துதல் என்று சொல்வது என்ன?"))

தத்துவப்படுத்துதல் என்பது ஒரு நுண்ணறிவு செயலாகும். இதை நுண்சிந்தனை என்றும் அழைக்கலாம். இது அடிப்படையில் தர்க்கத்தை மேற்கொண்டு சிந்தனையை விரிவாக்குதல், புதியவற்றை உருவாக்குதல், மற்றும் முற்றிலும் புதிய தளத்திற்கு தாவிச் செல்லுதல் போன்ற செயல்களை உள்ளடக்கியது. இதன் மூலம், நமக்கு அளிக்கப்பட்ட தகவல்களில் உள்ள தர்க்கத்தை பயன்படுத்தி, அச்சிந்தனையை நுண்ணறிவுடன் விரிவாக்க முடியும். 

இது ஒரு மூன்றடுக்கு செயலாகும்:
1. தர்க்கம் - அளிக்கப்பட்டவற்றில் உள்ள தர்க்கத்தை மேற்கொண்டு முன்னெடுத்து விரிவாக்குதல்.
2. கற்பனை - புதியவற்றை உருவாக்குதல்.
3. நுண்ணுணர்வு - முற்றிலும் புதிய தளத்திற்கு தாவிச் செல்லுதல்.

தத்துவ வகுப்புகளில் இந்நுண்சிந்தனையை அல்லது தத்துவப்படுத்தலுக்கு பயிற்சி அளிக்கப்படுகிறது.


In [8]:
print(rag_chain.invoke("நம்முடைய கல்லூரிகளின் நிலை என்ன?"))

நம்முடைய கல்லூரிகளின் நிலை மிகுந்த சிரமமானதாகும். மாணவர்கள் குறிப்பாக புத்தகங்களில் உள்ளவற்றை நினைவில் கொண்டு எழுதுவதில் மட்டுமே திறமைசாலிகள். சுயசிந்தனை, சுருக்கம் அல்லது சாராம்சப்படுத்தல் போன்ற திறன்கள் கற்றுக்கொடுக்கப்படுவதில்லை. கல்விமுறையின் அடிப்படை சிக்கல் இது. 

மாணவர்கள் வேலைக்குச் சென்றால், மூளையுழைப்புக்கு தயாரானவர்களாக இருப்பதற்காக, சலிப்பூட்டும் வேலைகளைச் செய்யும் மனித இயந்திரங்களாக மாறுகிறார்கள். இது அவர்களுக்கு வேலை கிடைப்பதற்கான காரணமாகும். 

கல்விமுறை ஆசிரியர்கள் மதிப்பெண் வழங்கும்போது வெளிப்படைத்தன்மையை பேணுவதில் கவனம் செலுத்துகின்றனர். மாணவர்கள் சுயமாக சிந்திக்கக் கற்றுக்கொள்ளுவதில்லை என்பதே இதற்குக் காரணம். 

அதனால், மாணவர்களுக்கு சிந்தனை, சாராம்சப்படுத்தல், நுண்சிந்தனை போன்றவற்றை கற்றுக்கொடுக்க வேண்டும். இவை இல்லையெனில், அவர்கள் எதிர்காலத்தில் மனித மூளையுழைப்பைச் செய்ய முடியாது, ஏனெனில் மற்ற அனைத்தையும் செயற்கை நுண்ணறிவு செய்யும். 

எனவே, நம் கல்லூரிகளின் நிலை மிகுந்த சிக்கலானதாக உள்ளது.


In [12]:
print(rag_chain.invoke("திருக்குறள் பற்றி கூறுக."))

திருக்குறள் பற்றி தரப்பட்டுள்ள தகவல் எதுவும் கிடைக்கவில்லை.
